# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *


In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [8]:
img_size=128
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [9]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20201213_201451_cgan_long_run_model4', '20201214_132…

In [12]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201214_121039_lambda1.0


In [13]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'128':raw_data_dir+'128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
                 '512':raw_data_dir+'512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'}

## Plot Losses

In [27]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [28]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
62491,62491.0,19.0,0.087119,0.100559,0.187678,5.259452,82.613022,77.353569,-2.146474,83.064354,-2.232133,4.105299,-4.489694,-5.245277,0.131732
62492,62492.0,19.0,0.092735,0.045265,0.138000,4.921316,82.782196,77.860878,-2.087796,91.425240,-1.979829,4.098468,-4.241503,-4.896261,0.133286
62493,62493.0,19.0,0.053473,0.082304,0.135777,4.567583,82.851616,78.284035,-1.950125,90.258301,-1.767209,4.001629,-4.649742,-4.535023,0.133238
62494,62494.0,19.0,0.120312,0.169187,0.289499,6.563428,84.581100,78.017670,-2.072094,86.104034,-2.271501,4.741310,-2.834411,-6.560762,0.133630
62495,62495.0,19.0,0.274408,0.120959,0.395367,3.495410,82.792374,79.296967,-2.013377,87.590500,-1.922777,2.211231,-6.593904,-3.427736,0.133329
62496,62496.0,19.0,0.110232,0.269645,0.379878,6.522943,84.288826,77.765884,-2.021137,79.344177,-1.906934,5.378328,-2.318744,-6.520598,0.133718
62497,62497.0,19.0,0.090134,0.054066,0.144200,6.456260,85.662338,79.206078,-2.106754,89.117378,-1.726409,3.643576,-5.669875,-6.447915,0.132948
62498,62498.0,19.0,0.118277,0.083921,0.202198,4.085314,86.062019,81.976707,-1.819205,82.978760,-2.038397,3.603843,-5.758281,-4.046115,0.132877
62499,62499.0,19.0,0.138040,0.127048,0.265088,6.071375,81.744926,75.673553,-2.012068,80.620934,-2.034742,5.847179,-3.128812,-6.064336,0.134583
62500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.804962,-1.623186,NaN,NaN,NaN,NaN


In [29]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [17]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-3.038047105073929


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
61182,61182.0,19.0,0.063793,0.184135,0.247928,5.736397,84.266312,78.529915,-3.056120,87.073631,-3.255671,5.341962,-3.580693,-5.727769,0.130701
57997,57997.0,18.0,0.143000,0.082259,0.225259,5.650477,82.252747,76.602272,-2.867628,85.787025,-3.238164,5.843637,-4.097420,-5.641694,0.138960
59198,59198.0,18.0,0.079168,0.100696,0.179864,5.042528,87.847206,82.804680,-2.538407,84.064690,-3.228681,3.875629,-3.381148,-5.026120,0.130923
58017,58017.0,18.0,0.241451,0.397206,0.638657,12.065363,92.266296,80.200935,-3.136019,85.965294,-3.225352,13.214107,-1.662415,-12.065351,0.136314
59201,59201.0,18.0,0.856564,0.102584,0.959148,1.450821,79.324623,77.873802,-2.603203,83.591660,-3.204951,-0.061176,-8.744335,-0.878811,0.134032
60985,60985.0,19.0,0.097011,0.106277,0.203288,6.307673,84.680092,78.372421,-3.043886,77.413818,-3.204796,7.203027,-4.855757,-6.301249,0.136368
58019,58019.0,18.0,0.288267,0.164366,0.452633,9.223439,88.257599,79.034157,-3.179496,79.998230,-3.191111,2.263774,-11.950095,-9.223066,0.137093
58020,58020.0,18.0,0.082393,0.139519,0.221912,5.657308,82.479973,76.822662,-3.228936,84.181305,-3.188744,5.349707,-7.607990,-5.645276,0.138946
60578,60578.0,19.0,0.099059,0.091882,0.190941,5.989990,85.210365,79.220375,-3.072991,84.764999,-3.186998,7.323106,-5.791258,-5.983612,0.144953
58024,58024.0,18.0,0.185026,0.108236,0.293262,4.673256,82.302528,77.629272,-3.206500,84.449112,-3.186661,3.382659,-5.933995,-4.632154,0.137057


In [18]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
61182,61182.0,19.0,0.063793,0.184135,0.247928,5.736397,84.266312,78.529915,-3.056120,87.073631,-3.255671,5.341962,-3.580693,-5.727769,0.130701
57997,57997.0,18.0,0.143000,0.082259,0.225259,5.650477,82.252747,76.602272,-2.867628,85.787025,-3.238164,5.843637,-4.097420,-5.641694,0.138960
59198,59198.0,18.0,0.079168,0.100696,0.179864,5.042528,87.847206,82.804680,-2.538407,84.064690,-3.228681,3.875629,-3.381148,-5.026120,0.130923
58017,58017.0,18.0,0.241451,0.397206,0.638657,12.065363,92.266296,80.200935,-3.136019,85.965294,-3.225352,13.214107,-1.662415,-12.065351,0.136314
59201,59201.0,18.0,0.856564,0.102584,0.959148,1.450821,79.324623,77.873802,-2.603203,83.591660,-3.204951,-0.061176,-8.744335,-0.878811,0.134032
60985,60985.0,19.0,0.097011,0.106277,0.203288,6.307673,84.680092,78.372421,-3.043886,77.413818,-3.204796,7.203027,-4.855757,-6.301249,0.136368
58019,58019.0,18.0,0.288267,0.164366,0.452633,9.223439,88.257599,79.034157,-3.179496,79.998230,-3.191111,2.263774,-11.950095,-9.223066,0.137093
58020,58020.0,18.0,0.082393,0.139519,0.221912,5.657308,82.479973,76.822662,-3.228936,84.181305,-3.188744,5.349707,-7.607990,-5.645276,0.138946


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
21611,21611.0,6.0,0.152790,0.163282,0.316072,3.732631,83.715279,79.982651,-2.822460,73.642944,-2.715360,3.688989,-4.114289,-3.671987,0.137162
22422,22422.0,7.0,0.157858,0.414247,0.572105,5.938013,89.422142,83.484131,-2.836155,73.874741,-2.698003,5.226918,-1.697260,-5.933554,0.130813
32538,32538.0,10.0,0.091417,0.236511,0.327928,5.560940,85.534203,79.973259,-2.403343,74.211365,-2.409706,5.245386,-2.569450,-5.553817,0.135217
20612,20612.0,6.0,0.222665,0.064308,0.286973,4.522329,85.921921,81.399590,-2.477599,74.287468,-2.543133,2.519631,-6.664237,-4.480016,0.132927
22181,22181.0,7.0,0.117515,0.160824,0.278339,4.097977,82.441628,78.343651,-2.629564,74.310364,-2.409084,3.699951,-2.838430,-4.062719,0.129456
21104,21104.0,6.0,0.090012,0.150071,0.240083,4.665524,85.404282,80.738754,-2.372139,74.397858,-2.180544,3.924229,-2.555491,-4.640551,0.134157
26160,26160.0,8.0,0.191015,0.124005,0.315020,2.592765,82.986870,80.394104,-2.934070,74.416153,-2.667293,3.039682,-3.122101,-2.463003,0.131857
26150,26150.0,8.0,0.276871,0.104284,0.381155,2.240471,78.981850,76.741379,-2.975537,74.483398,-2.985421,2.120606,-3.694033,-2.033417,0.129740


## Read stored chi-squares for images

In [16]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [17]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [18]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [19]:
df_sliced=w.result
# df_sliced

In [20]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_1',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False).step.values)
print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 0.05342877707330067, 'chi_spec1': 0.6504270204692923}
[0, 19]
{'chi_1': 0.19627089923781835, 'chi_spec1': 1.9439437099354455}
[0, 19]
{'chi_1': 0.05342877707330067, 'chi_spec1': 0.6504270204692923}
[array([47070, 55860]), array([52700, 10460]), array([32880, 33230])]
[10460 32880 33230 47070 52700 55860]


In [21]:
# best_step=[58800]
# best_step=np.arange(32800,32900,10).astype(int)

In [22]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(6, 22)
[(3, 10460), (10, 32880), (10, 33230), (15, 47070), (16, 52700), (17, 55860)]


In [23]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
1045,3-10460,0.073113,1.323515,34.248018,0.060882,0.011178,0.001053,0.037061,17.846738,40.685245,3,10460
3287,10-32880,0.010184,0.064247,28.584885,0.002682,0.005782,0.001720,0.000626,17.705419,2.461963,10,32880
3322,10-33230,0.007975,0.082586,28.110264,0.003664,0.002857,0.001455,0.001032,15.936214,6.226584,10,33230
4706,15-47070,0.005451,0.378988,27.943169,0.000530,0.003725,0.001196,0.000369,14.025280,29.227187,15,47070
5269,16-52700,0.023294,0.368052,28.927640,0.012854,0.009394,0.001045,0.009556,16.587920,26.082959,16,52700
5585,17-55860,0.005669,0.488279,28.748102,0.000950,0.002466,0.002253,0.000216,13.983533,40.135026,17,55860


### Plots 2: stored data

In [24]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

(1000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [25]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')



In [26]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('3-10460', '10-32880', '10-33230', '1…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [31]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(ip_fname,mmap_mode='r')[:1000,0,:,:]
print(s_val.shape)
f_plot_grid(s_val[100:118],cols=6,fig_size=(6,3))



(1000, 128, 128)
3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …